In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# plt.rcParams['figure.dpi'] = 100

## Methods
### Condensed sparse column matrices

In [ ]:
data = np.array([1, 2, 3, 4, 5, 6])
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
sp.sparse.csc_matrix((data, (row, col)), shape=(3, 3)).toarray()

In [ ]:
# topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
#           'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
#           'energy', 'optics', 'earth science', 'geology', 'meteorology']
topics = ['earth science']

In [ ]:
path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_saved + topic + '.pickle')
graph = networks[topic].graph

In [ ]:
len(networks[topic].graph.nodes)

In [ ]:
v = networks[topic].graph.graph['tfidf']
v

In [ ]:
v.sum()

In [ ]:
v[:,0].indices[:5]

In [ ]:
v[4,0]

In [ ]:
networks[topic].graph.name

In [ ]:
networks[topic].graph.nodes['Biology']

In [ ]:
core = [n for n in networks[topic].graph.nodes if networks[topic].graph.nodes[n]['core_rb']>.9]
core

In [ ]:
[(i,n) for i,n in enumerate(networks[topic].graph.nodes) if networks[topic].graph.nodes[n]['year']<-1800]

In [ ]:
vi = v[:,9]
vi

### CSC & networkx operations

In [ ]:
graph = networks[topic].graph

In [ ]:
core = [n for n in networks[topic].graph.nodes if networks[topic].graph.nodes[n]['year']<-2000]
subgraph = graph.subgraph(core).copy()

In [ ]:
import scipy.sparse as ss

In [ ]:
tfidf = ss.hstack([v[:,list(graph.nodes).index(n)] for n in subgraph.nodes])
tfidf

In [ ]:
subgraph.nodes

In [ ]:
subgraph.add_node('Hello')

In [ ]:
subgraph.nodes

## Algorithm

Initialize with core set of nodes.\
For each year,\
initialize an "baby" node for each existing node that doesn't already have a baby node,\
mutate tf-idf for each "baby" node (including the name),\
and if the "baby" node gets a probability drawn from the distribution of similarities (to what?),
the "baby" node is born.

In [ ]:
import sklearn.metrics.pairwise as smp
import scipy.sparse as ss
from scipy.stats import norm
import seaborn as sn

### Mutation

#### Prior: power law distributions of weights

In [ ]:
graph = networks[topic].graph
tfidf = graph.graph['tfidf'].copy()

In [ ]:
import powerlaw
fit = powerlaw.Fit(tfidf[:,1].data)
fit.plot_pdf()
fit.power_law.plot_pdf();
plt.title(f"xmin={fit.xmin:.1e}, α={fit.alpha:.1f}");

#### Prior: new words / year between neighbors
[gist](https://gist.github.com/ptocca/e18a9e4e35930c0958fdaa62958bdf82)

In [ ]:
def year_diffs(graph):
    return [graph.nodes[node]['year'] - graph.nodes[neighbor]['year']
            for node in graph.nodes
            for neighbor in list(graph.successors(node))]

yd = year_diffs(graph)
sns.distplot(yd)
plt.title(topic)
plt.xlabel('year difference');

In [ ]:
%reload_ext cython

In [ ]:
%%cython -f

import numpy as np
cimport numpy as np
from cython cimport floating,boundscheck,wraparound
from cython.parallel import prange

from libc.math cimport fabs

np.import_array()

@boundscheck(False)  # Deactivate bounds checking
@wraparound(False)
def cython_manhattan(floating[::1] X_data, int[:] X_indices, int[:] X_indptr,
                     floating[::1] Y_data, int[:] Y_indices, int[:] Y_indptr,
                     double[:, ::1] D):
    """Pairwise L1 distances for CSR matrices.
    Usage:
    >>> D = np.zeros(X.shape[0], Y.shape[0])
    >>> cython_manhattan(X.data, X.indices, X.indptr,
    ...                  Y.data, Y.indices, Y.indptr,
    ...                  D)
    """
    cdef np.npy_intp px, py, i, j, ix, iy
    cdef double d = 0.0
    
    cdef int m = D.shape[0]
    cdef int n = D.shape[1]
    
    with nogil:                          
        for px in prange(m):
            for py in range(n):
                i = X_indptr[px]
                j = Y_indptr[py]
                d = 0.0
                while i < X_indptr[px+1] and j < Y_indptr[py+1]:
                    if i < X_indptr[px+1]: ix = X_indices[i]
                    if j < Y_indptr[py+1]: iy = Y_indices[j]
                    
                    if ix==iy:
                        d = d+fabs(X_data[i]-Y_data[j])
                        i = i+1
                        j = j+1
                    
                    elif ix<iy:
                        d = d+fabs(X_data[i])
                        i = i+1
                    else:
                        d = d+fabs(Y_data[j])
                        j = j+1
                
                if i== X_indptr[px+1]:
                    while j < Y_indptr[py+1]:
                        iy = Y_indices[j]
                        d = d+fabs(Y_data[j])
                        j = j+1                                            
                else:
                    while i < X_indptr[px+1]:
                        ix = X_indices[i]
                        d = d+fabs(X_data[i])
                        i = i+1
                        
                D[px,py] = d

In [ ]:
import sklearn.preprocessing as skp
import sklearn.metrics.pairwise as smp
from scipy.sparse import csr_matrix,random
from sklearn.metrics.pairwise import check_pairwise_arrays

def sparse_manhattan(X,Y=None):
    X, Y = check_pairwise_arrays(X, Y)
    X = csr_matrix(X, copy=False)
    Y = csr_matrix(Y, copy=False)
    res = np.empty(shape=(X.shape[0],Y.shape[0]))
    cython_manhattan(X.data,X.indices,X.indptr,
                     Y.data,Y.indices,Y.indptr,
                             res)
    return res

def word_diffs(graph, tfidf):
    dists = sparse_manhattan(X=skp.binarize(tfidf).transpose())
    nodes = list(graph.nodes)
    return [dists[nodes.index(node), nodes.index(neighbor)]
            for node in nodes
            for neighbor in list(graph.successors(node))]

plt.figure(figsize=(14,5))
plt.subplot(121)
wd = word_diffs(graph, tfidf)
sns.scatterplot(x=np.abs(yd), y=wd)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), wd)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e}")
plt.xlabel('year')
plt.ylabel('manhattan distance');

plt.subplot(122)
sns.distplot(wd)
mu, std = sp.stats.norm.fit(wd)
x = np.linspace(min(wd), max(wd), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.xlabel('manhattan distance')
plt.ylabel('probability distribution');

#### Prior: similarity / year between neighbors

In [ ]:
def neighbor_similarity(graph, tfidf):
    nodes = list(graph.nodes)
    return [smp.cosine_similarity(tfidf[:,nodes.index(node)].transpose(),
                                  tfidf[:,nodes.index(neighbor)].transpose())[0,0]
            for node in nodes
            for neighbor in list(graph.successors(node))]

neighbors = neighbor_similarity(graph, tfidf)
plt.figure(figsize=(6,4))
sns.scatterplot(x=np.abs(yd), y=neighbors)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), neighbors)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e}")
plt.xlabel('Δyear')
plt.ylabel('cosine similarity');

#### Prior: weight distributions of nodes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_distribution(data):
    bins = np.logspace(np.log10(min(data)), np.log10(max(data)), 50)
    hist, edges = np.histogram(data, bins=bins)
#     hist_norm = hist/(bins[1:] - bins[:-1])
    sns.scatterplot(bins[:-1], hist/len(data))
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim(bins[0]/2, bins[-1]*2)
    plt.ylim(min(hist[hist>0])/len(data)/2, 1)
    plt.xlabel('x')
    plt.ylabel('P(x)')

plt.figure(figsize=(14,6))

plt.subplot(121)
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': tfidf.indices,
                                   'weight': tfidf.data}))
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': tfidf.indices,
                                   'weight': tfidf.data})\
                       .groupby('index').mean()\
                       .reset_index())
plt.ylim([-.2,1.2]);

plt.subplot(122)
plot_distribution(tfidf.data)

#### Prior: year distribution

In [ ]:
sns.distplot([graph.nodes[node]['year'] for node in graph.nodes], rug=True)
plt.xlabel('year');

#### Method

In [ ]:
import numpy.random as npr

def mutate(x, rvs, point=(0,0), insert=(0,0,None), delete=(0,0)):
    """ Mutates vector ``x`` with point mutations,
    insertions, and deletions. Insertions and point
    mutations draw from a random process ``rvs``.
    
    Parameters
    ----------
    x: spipy.sparse.csc_matrix
    rvs: lambda ()-> float
        returns a random weight, [0,1]
    point: tuple (int n, float p)
        n = number of elements to insert
        p = probability of insertion for each trial
    insert: tuple (n, p, iterable s)
        s = set of elements from which to select
            if None, select from all zero elements
    delete: tuple (n, p)
    """
    data = x.data
    idx = x.indices
    for _ in range(point[0]):
        if npr.rand() < point[1]:
            data[npr.choice(x.size)] = rvs()
    for _ in range(insert[0]):
        if npr.rand() < insert[1]:
            while True:
                insert_idx = npr.choice(insert[2]) if insert[2]\
                    else npr.choice(x.shape[0])
                if insert_idx not in idx: break
            idx = np.append(idx, insert_idx)
            data = np.append(data, rvs())
    for _ in range(delete[0]):
        if npr.rand() < delete[1]:
            delete_idx = npr.choice(idx.size)
            idx = np.delete(idx, delete_idx)
            data = np.delete(data, delete_idx)
    y = ss.csc_matrix((data, (idx, np.zeros(idx.shape, dtype=int))),
                      shape=x.shape)
    return y

#### Test

In [ ]:
x = tfidf[:,1].copy()
y = tfidf[:,1].copy()
T = 300

sim = np.zeros(T)
size = np.zeros(T)
for i in range(sim.size):
    sim[i] = smp.cosine_similarity(x.transpose(),y.transpose())[0,0]
    size[i] = y.size
    y = mutate(y, lambda: fit.power_law.generate_random()[0],
               point=(10,.5), insert=(10,.5,None), delete=(10,.5))

plt.figure(figsize=(14,4))
plt.subplot(121)
sn.lineplot(x=range(sim.size), y=sim)
plt.title(graph.name)
plt.ylabel('similarity')
plt.xlabel('years');
plt.subplot(122)
sn.lineplot(x=range(sim.size), y=size)
plt.title(graph.name)
plt.ylabel('size')
plt.xlabel('years');

plt.figure()
plot_distribution(graph.graph['tfidf'][:,1].data)
plt.xlabel('tf-idf values');
plot_distribution(y.data)
plt.title(graph.name)
plt.legend(['before mutation', 'after mutation'])
plt.xlabel('tf-idf values');

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(121)
plot_distribution(x.data)
plot_distribution(y.data)
plt.legend(['before','after']);

plt.subplot(122)
plot_distribution(x.data)
plot_distribution(y.data)
plt.yscale('linear')
plt.xscale('linear')
plt.ylim([0,.2])
plt.xlim([0,.1])
plt.legend(['before','after']);

### Create new nodes

#### Prior: distribution of similarities

In [ ]:
def non_neighbor_similarity(graph, tfidf):
    nodes = list(graph.nodes)
    sim = [smp.cosine_similarity(tfidf[:,nodes.index(n1)].transpose(),
                                 tfidf[:,nodes.index(n2)].transpose())[0,0]
           for n1 in graph.nodes
           for n2 in graph.nodes
           if (n2 is not n1) and (n2 not in list(graph.neighbors(n1)))]
    return sim

non_neighbors = non_neighbor_similarity(graph, tfidf)

plt.figure()
sns.distplot(neighbors)
x = np.linspace(min(neighbors), max(neighbors), 100)
mu, std = sp.stats.norm.fit(neighbors)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
sns.distplot(non_neighbors)
plt.title(topic)
plt.legend([f"fit-neighbors (m={mu:.2f}; s={std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity');

#### Method

Just draw from normal pdf

#### Test

In [ ]:
npr.normal(loc=mu, scale=std, size=4)

### Crossover

What prior should I use? It needs to be more similar than neighbors. Some kind of a t-test?

#### Prior: maybe just 3 std above mean?

In [ ]:
mu + 3*std

#### Method

average? or combine elements?

In [ ]:
def crossover(v1, v2):
    """ Crosses two vectors by combining half of one
    and half of the other.
    
    Parameters
    ----------
    v1, v2: scipy.sparse.matrix
    
    Returns
    -------
    v3: scipy.sparse.matrix
    """
    idx1 = npr.choice(v1.size, size=int(v1.size/2))
    idx2 = npr.choice(v2.size, size=int(v2.size/2))
    data = np.array([v1.data[i] for i in idx1] +
                    [v2.data[i] for i in idx2])
    idx = np.array([v1.indices[i] for i in idx1] +
                   [v2.indices[i] for i in idx2])
    v3 = ss.csc_matrix((data, (idx, np.zeros(idx.shape,
                                             dtype=int))),
                       shape=v1.shape)
    return v3

def crossover_seeds(seeds, threshold=.7):
    """ Crosses ``seeds`` if similarity between two seeds
    is greater than ``threshold``. Then, it sets one of the
    seeds to ``None``.
    
    Parameters
    ----------
    seeds: dict {str: scipy.sparse.csc_matrix}
    threshold: float
    """
    nodes = list(seeds.keys())
    for i in range(len(nodes)):
        for j in range(i+1,len(nodes)):
            if nodes[i] not in seeds.keys() or nodes[j] not in seeds.keys():
                continue
            similarity = smp.cosine_similarity(seeds[nodes[i]].transpose(),
                                               seeds[nodes[j]].transpose())
            if similarity[0,0] > threshold:
                seeds[nodes[i]] = crossover(seeds[nodes[i]], seeds[nodes[j]])
                seeds.pop(nodes[j], None)

#### Test

In [ ]:
tfidf = graph.graph['tfidf'].copy()
nodes = list(graph.nodes)[:6]
seeds = {node: tfidf[:,list(graph.nodes).index(node)]
         for node in nodes}
seeds

In [ ]:
vectors = ss.hstack([seeds[node] for node in nodes])
print(np.triu(smp.cosine_similarity(vectors.transpose())))
crossover_seeds(seeds, threshold=0.5)
print('----------------------------------------------------------')
vectors = ss.hstack([seeds[node] for node in nodes if node in seeds.keys()])
print(np.triu(smp.cosine_similarity(vectors.transpose())))

#### Method

In [ ]:
def consume_seeds(seeds, vectors, threshold=0.9):
    """ Consumes a seed in ``seeds`` if similarity
    between a seed and an existing vector in ``vectors``
    is greater than ``threshold``.
    
    Parameters
    ----------
    seeds: dict {string: scipy.sparse.csc_matrix}
    vectors: scipy.sparse.csc_matrix
    threshold: float
    """
    for seed, vec in list(seeds.items()):
        for i in range(vectors.shape[1]):
            s = smp.cosine_similarity(vec.transpose(), vectors[:,i].transpose())
            if s[0,0] > threshold:
                seeds.pop(seed, None)

#### Test

In [ ]:
tfidf = graph.graph['tfidf'].copy()
nodes = list(graph.nodes)[:4]
seeds = {node: tfidf[:,list(graph.nodes).index(node)]
         for node in nodes}
T = 100
seeds

In [ ]:
for _ in range(10):
    seeds['Hydrosphere'] = mutate(seeds['Hydrosphere'],
                                  lambda: fit.power_law.generate_random()[0],
                                  point=(1,1), insert=(5,.5,None), delete=(5,.5))
consume_seeds(seeds, tfidf[:,:4])
seeds

### Create nodes

#### Get words from tf-idf vector

In [ ]:
import pickle
import gensim.utils as gu

path_models = '/Users/harangju/Developer/data/wiki/models/'
model = gu.SaveLoad.load(path_models + 'tfidf.model')
dct = pickle.load(open(path_models + 'dict.model','rb'))

In [ ]:
words = [dct[i] for i in tfidf[:,0].indices]
words[:5]

#### Prior: word weight vs title

In [ ]:
idx = np.argsort(tfidf[:,0].data)
idx[-5:], tfidf[:,0].data[idx[-10:]]

In [ ]:
def find_top_words(x, dct, top_n=5, stoplist=set('for a of the and to in'.split())):
    """
    
    Parameters
    ----------
    x: scipy.sparse.csc_matrix
    dct: gensim.corpora.dictionary
    top_n: int
    
    Returns
    -------
    words:
    idx_vector: 
    """
    top_idx = np.argsort(x.data)[-top_n:]
    idx = [x.indices[i] for i in top_idx if dct[x.indices[i]] not in stoplist]
    words = [dct[i] for i in idx]
    return words, idx

In [ ]:
stoplist=set('for a of the and to in'.split())
nodes = []
words1 = []
words2 = []
for i in range(tfidf.shape[1]):
    if tfidf[:,i].data.size == 0:
        print(list(graph.nodes)[i], tfidf[:,i].data)
        continue
    nodes += [list(graph.nodes)[i]]
    idx_sorted = np.argsort(tfidf[:,i].data)
    words1 += [[dct[tfidf[:,i].indices[idx]]
                for idx in idx_sorted[-5:]
                if dct[tfidf[:,i].indices[idx]] not in stoplist]]
    top_words, idx = find_top_words(tfidf[:,i], dct, top_n=5)
    words2 += [top_words]
pd.DataFrame(data={'Node': nodes, 'Top words 1': words1, 'Top words 2': words2})

#### Method

If article has any two of the top five words, connect.
```
for new_article in new_articles:
    for article in articles:
        if any two of the top five words are in new_article:
            connect new_article to article
```

In [ ]:
def connect(seed_vector, graph, vectors, dct, top_words=5, match_n=2):
    """
    
    Parameters
    ----------
    seed_vector: scipy.sparse.csc_matrix
    graph: networkx.DiGraph (not optional)
    vectors: scipy.sparse.csc_matrix (not optional)
    dct: gensim.corpora.dictionary (not optional)
    top_words: int (default=5)
    match_n: int
        how many words should be matched by...
    """
    seed_top_words, seed_top_idx = find_top_words(seed_vector, dct)
    seed_name = ' '.join(seed_top_words)
    nodes = list(graph.nodes)
    graph.add_node(seed_name)
    for i, node in enumerate(nodes):
        node_vector = vectors[:,i]
        node_top_words, node_top_idx = find_top_words(node_vector, dct)
        if len(set(seed_top_idx).intersection(set(node_vector.indices))) >= match_n:
            graph.add_edge(node, seed_name)
        if len(set(node_top_idx).intersection(set(seed_vector.indices))) >= match_n:
            graph.add_edge(seed_name, node)

#### Test

In [ ]:
graph = networks[topic].graph
core_nodes = [n for n in graph.nodes if graph.nodes[n]['year'] < -2000]
subgraph = graph.subgraph(core_nodes).copy()
subgraph.graph.clear()
subgraph.name = graph.name + '-cutting'
print(f"Core nodes: {core_nodes} in '{subgraph.name}'")

In [ ]:
test_graph = subgraph.copy()
test_vector = ss.hstack([tfidf[:,list(graph.nodes).index(n)] for n in test_graph.nodes])

seed = 'Meteorology'
seed_vector = tfidf[:,list(graph.nodes).index(seed)]

print('Nodes:', test_graph.nodes)
print('Edges:', test_graph.edges, '\n')
print(f"Seed: {seed}\n")
connect(seed_vector, test_graph, test_vector, dct, match_n=3)
print('Nodes:', test_graph.nodes)
print('Edges:', test_graph.edges)

#### Test with all nodes without node names

### Evolve

#### Priors

In [ ]:
# import powerlaw
# fit = powerlaw.Fit(graph.graph['tfidf'].data)
fit.plot_pdf()
fit.power_law.plot_pdf();
plt.title(f"Power law x_min={fit.xmin:.1e}, α={fit.alpha:.1f}");

In [ ]:
sns.scatterplot(x=np.abs(yd), y=wd)
slope, intercept, fit_r, p, stderr = sp.stats.linregress(np.abs(yd), wd)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={fit_r:.2f}; p={p:.1e}")
plt.xlabel('Δyear')
plt.ylabel('manhattan distance (# different words)');

In [ ]:
neighbors = neighbor_similarity(graph, tfidf)
fit_mu, fit_std = sp.stats.norm.fit(neighbors)
non_neighbors = non_neighbor_similarity(graph, tfidf)
sns.distplot(neighbors)
x = np.linspace(min(neighbors), max(neighbors), 100)
plt.plot(x, sp.stats.norm.pdf(x, fit_mu, fit_std))
sns.distplot(non_neighbors)
plt.title(topic)
plt.legend([f"fit-neighbors (m={fit_mu:.2f}; s={fit_std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity');

In [ ]:
fit_mu + 3*fit_std

#### Method
1. Initialize a bag of seeds from a set of nodes.
2. For each year,
    1. Mutate seeds. For each seed,
        1. Change a word with `p_point`. Draw weight from power law prior.
        2. Delete a word with `p_delete`.
        3. Insert new word with `p_insert`. Draw weight from power law prior.
    2. Crossover seeds if `μ+3σ < similarity`.
    3. Create new node from seed if `x < similarity` where `x~Norm(θ)`.
        1. Connect new node.
        2. Initialize new seed.

In [ ]:
import sys

def initialize_seeds(seeds, graph, vectors, thresholds, neighbor_stats):
    for node in graph.nodes:
        if node not in seeds.keys():
            seeds[node] = vectors[:,list(graph.nodes).index(node)].copy()
            thresholds[node] = npr.normal(loc=neighbor_stats[0],
                                          scale=neighbor_stats[1])

def mutate_seeds(seeds, rvs, point, insert, delete):
    for node, vec in seeds.items():
        seeds[node] = mutate(vec, rvs, point=point, insert=insert, delete=delete)

def create_nodes(seeds, graph, vectors, thresholds, year):
    nodes = list(graph.nodes)
    for node in nodes:
        sim_to_parent = smp.cosine_similarity(seeds[node].transpose(),
                                              vectors[:,nodes.index(node)].transpose())
        if sim_to_parent[0,0] < thresholds[node]:
            connect(seeds[node], graph, vectors, dct, match_n=3)
            vectors = ss.hstack([vectors, seeds[node]])
            seeds.pop(node)
    for node in graph.nodes:
        if 'year' not in graph.nodes[node].keys():
            graph.nodes[node]['year'] = year
    return vectors

def evolve(graph, vectors, year_end, rvs, point, insert, delete, neighbor_stats):
    """ Evolves a graph based on vector representations
    
    Parameters
    ----------
    graph: networkx.DiGraph
    vectors: scipy.sparse.csc_matrix
    year_end: int (default=2020)
    rvs: lambda: float
    point, insert, delete: tuple
        See ``mutate()``.
    neighbor_stats: tuple of floats
        (mu, std)
    """
    year_start = max([graph.nodes[n]['year'] for n in graph.nodes])+1
    seeds = {}
    thresholds = {}
    data = pd.DataFrame()
    for year in range(year_start, year_end+1):
        sys.stdout.write('\r                                     ')
        sys.stdout.write(f"\r{year_start}\t> {year}\t> {year_end}")
        sys.stdout.flush()
        initialize_seeds(seeds, graph, vectors, thresholds, neighbor_stats)
        mutate_seeds(seeds, rvs, point=point, insert=insert, delete=delete)
        vectors = create_nodes(seeds, graph, vectors, thresholds, year)
        crossover_seeds(seeds, neighbor_stats[0]+3*neighbor_stats[1])
        for seed, vector in seeds.items():
            data = data.append({'Year': year,
                                'Parent': seed,
                                'Seed vectors': vector},
                               ignore_index=True)
    return vectors, data

#### Test

In [ ]:
start_year = -500
core_nodes = [n for n in graph.nodes if graph.nodes[n]['year'] < start_year]
subgraph = graph.subgraph(core_nodes).copy()
subgraph.graph.clear()
tfidf = graph.graph['tfidf']
vectors = ss.hstack([tfidf[:,list(graph.nodes).index(n)] for n in core_nodes])
print(f"Topic: '{graph.name}'" +\
      f"Core nodes: {core_nodes}" +\
      f"Parameters:\tα (power law): {fit.alpha:.2f}\n\t\t" +\
      f"p_insert/delete: {fit_r:.2f}\n\t\t" +\
      f"neighbor_mu, std: {fit_mu:.2f}, {fit_std:.2f}\n\t\t" +\
      f"threshold: {fit_mu+3*fit_std:.2f}")

In [ ]:
vectors, data = evolve(subgraph, vectors,
                       year_end=2000,
                       rvs=lambda: fit.power_law.generate_random()[0],
                       point=(1,.5),
                       insert=(1,r,list(set(tfidf.indices))),
                       delete=(1,r),
                       neighbor_stats=(fit_mu,fit_std))
print('\n'+repr(vectors))
data

#### Compare priors

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(121)
sns.distplot(neighbors)
x = np.linspace(min(neighbors), max(neighbors), 100)
mu, std = sp.stats.norm.fit(neighbors)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
sns.distplot(non_neighbors)
plt.title(topic + ' (prior)')
plt.legend([f"fit-neighbors (m={mu:.2f}; s={std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity');
plt.xlim([-.2,1.2])
plt.subplot(122)
neighbors_model = neighbor_similarity(subgraph, vectors)
non_neighbors_model = non_neighbor_similarity(subgraph, vectors)
sns.distplot(neighbors_model)
x = np.linspace(min(neighbors_model), max(neighbors_model), 100)
mu, std = sp.stats.norm.fit(neighbors_model)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
sns.distplot(non_neighbors_model)
plt.title(topic + ' (model)')
plt.legend([f"fit-neighbors (m={mu:.2f}; s={std:.2f})", 'neighbors', 'non-neighbors'])
plt.xlabel('cos similarity')
plt.xlim([-.2,1.2]);

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
fit.plot_pdf()
fit.power_law.plot_pdf()
plt.title(f"empirical xmin={fit.xmin:.1e}, α={fit.alpha:.1f}");
plt.subplot(122)
fit_model = powerlaw.Fit(vectors.data)
fit_model.plot_pdf()
fit_model.power_law.plot_pdf()
plt.title(f"model xmin={fit_model.xmin:.1e}, α={fit_model.alpha:.1f}");

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(121)
sns.distplot(yd)
plt.title(topic + ' prior')
plt.xlabel('year difference')
plt.subplot(122)
yd_model = year_diffs(subgraph)
sns.distplot(yd_model)
plt.title(topic + ' model')
plt.xlabel('year difference');

In [ ]:
plt.figure(figsize=(14,10))
plt.subplot(221)
sns.scatterplot(x=np.abs(yd), y=wd)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd), wd)
x = np.linspace(0, max(yd), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (prior)")
plt.xlabel('year')
plt.ylabel('manhattan distance');

plt.subplot(222)
sns.distplot(wd)
mu, std = sp.stats.norm.fit(wd)
x = np.linspace(min(wd), max(wd), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.xlabel('manhattan distance')
plt.ylabel('probability distribution');
plt.title(f"μ={mu:.2}, σ={std:.2} (prior)")

wd_model = word_diffs(subgraph, vectors)

plt.subplot(223)
sns.scatterplot(x=np.abs(yd_model), y=wd_model)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd_model), wd_model)
x = np.linspace(0, max(yd_model), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e} (model)")
plt.xlabel('year')
plt.ylabel('manhattan distance');

plt.subplot(224)
sns.distplot(wd_model)
mu, std = sp.stats.norm.fit(wd_model)
x = np.linspace(min(wd_model), max(wd_model), 100)
plt.plot(x, sp.stats.norm.pdf(x, mu, std))
plt.xlabel('manhattan distance')
plt.ylabel('probability distribution');
plt.title(f"μ={mu:.2}, σ={std:.2} (model)");

In [ ]:
neighbors_model = neighbor_similarity(subgraph, vectors)

plt.figure(figsize=(6,4))
sns.scatterplot(x=np.abs(yd_model), y=neighbors_model)
slope, intercept, r, p, stderr = sp.stats.linregress(np.abs(yd_model), neighbors_model)
x = np.linspace(0, max(yd_model), 100)
sns.lineplot(x, np.multiply(slope, x) + intercept)
plt.title(f"slope={slope:.2f}; r={r:.2f}; p={p:.1e}")
plt.xlabel('Δyear')
plt.ylabel('cosine similarity');

In [ ]:
plt.figure(figsize=(14,6))

plt.subplot(121)
sns.scatterplot(x='index', y='weight',
                data=pd.DataFrame({'index': vectors.indices,
                                   'weight': vectors.data}))
plt.ylim([-.1,1.1]);

plt.subplot(122)
plot_distribution(vectors.data)

In [ ]:
plt.figure(figsize=(14,10))
plt.subplot(121)
nx.draw(graph)
plt.title('original graph')
plt.subplot(122)
nx.draw(subgraph)
plt.title('new graph');

In [ ]:
import json
%matplotlib inline

In [ ]:
def save_graph(g, name):
    nodes = [{'name': str(i)}#, 'club': 0 #g.node[i]['club']}
             for i in g.nodes()]
    links = [{'source': list(g.nodes).index(u),
              'target': list(g.nodes).index(v)}
             for u,v in g.edges()]
    with open(name + '.json', 'w') as f:
        json.dump({'nodes': nodes, 'links': links},
                  f, indent=4,)

save_graph(graph, 'graph')
save_graph(subgraph, 'subgraph')

In [ ]:
%%html
<div id="graph"></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [ ]:
%%html
<div id="subgraph"></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [ ]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths:
    {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
  // The code in this block is executed when the
  // d3.js library has been loaded.

  // First, we specify the size of the canvas
  // containing the visualization (size of the
  // <div> element).
  var width = 800, height = 400;
  var g = 'subgraph';

  // We create a color scale.
  var color = d3.scale.category10();

  // We create a force-directed dynamic graph layout.
  var force = d3.layout.force()
    .charge(-120)
    .linkDistance(50)
    .size([width, height]);

  // In the <div> element, we create a <svg> graphic
  // that will contain our interactive visualization.
  var svg = d3.select('#'.concat(g)).select("svg")
  if (svg.empty()) {
    svg = d3.select('#'.concat(g)).append("svg")
          .attr("width", width)
          .attr("height", height);
  }

  // We load the JSON file.
  d3.json(g.concat('.json'), function(error, graph) {
    // In this block, the file has been loaded
    // and the 'graph' object contains our graph.

    // We load the nodes and links in the
    // force-directed graph.
    force.nodes(graph.nodes)
      .links(graph.links)
      .start();

    // We create a <line> SVG element for each link
    // in the graph.
    var link = svg.selectAll(".link")
      .data(graph.links)
      .enter().append("line")
      .attr("class", "link");

    // We create a <circle> SVG element for each node
    // in the graph, and we specify a few attributes.
    var node = svg.selectAll(".node")
      .data(graph.nodes)
      .enter().append("circle")
      .attr("class", "node")
      .attr("r", 5)  // radius
      .style("fill", function(d) {
         // The node color depends on the club.
         return color(d.club);
      })
      .call(force.drag);

    // The name of each node is the node number.
    node.append("title")
        .text(function(d) { return d.name; });

    // We bind the positions of the SVG elements
    // to the positions of the dynamic force-directed
    // graph, at each time step.
    force.on("tick", function() {
      link.attr("x1", function(d){return d.source.x})
          .attr("y1", function(d){return d.source.y})
          .attr("x2", function(d){return d.target.x})
          .attr("y2", function(d){return d.target.y});

      node.attr("cx", function(d){return d.x})
          .attr("cy", function(d){return d.y});
    });
  });
});

### Discussion

The point of this model is that one can model knowledge discovery as incremental changes on existing knowledge.

The mutation model doesn't monotonically decrease similarity with parent.